<a href='https://colab.research.google.com/github/fernandovieira1/FinScore/blob/main/FINSCORE.ipynb' target='_parent'><img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/></a>

In [1]:
# Baseado em 'FINSCORE_v9.5.ipynb'

**# INTRUÇÕES**

* Antes de iniciar, certifique-se de estar logado na sua conta Google.

* Um botão azul 'Fazer login', localizado no canto superior direito da tela, aparecerá, se não.

* Feito isto, basta inserir os dados nas seções abaixo descritas:
    - 1.1 Cliente
    - 1.2 Período
    - 1.3 Score Serasa
    - 1.4 Lançamento dos dados Contábeis (na planilha indicada no link)

* Cada uma das seções mencionadas possui anotações sobre como proceder.

* Logo após, clique no menu 'Ambiente de execução' e em 'Executar tudo' (ou CTR + F9), nesta ordem.

#### 0. LANÇAMENTO DOS DADOS #####

##### 0.1 Cliente

*--> Insira o nome do cliente/empresa*

In [2]:
# Cliente
cliente = 'CARGOBR TRANSPORTES'

##### 0.2 Período

*--> Insira os anos inicial (a primeira) e final (da última) das demonstrações contábeis que serviram de base para a análise*

In [3]:
# Ano Inicial
ano_inicial = 2021

In [4]:
# Ano Final
ano_final = 2023

##### 0.3 Score Serasa

*--> Insira o score Serasa do cliente*

In [5]:
# Serasa
serasa = 550

##### 0.4 Lançamento dos dados Contábeis

*--> Acesse a planilha abaixo (CTRL + click) e insira as informações nas linhas e colunas respectivas*

In [6]:
# Lance do ano mais recente para o mais antigo
'https://docs.google.com/spreadsheets/d/1qx6nagvF8okAKvfO9BoHL2aZUZ8NOnz2/edit?gid=1575975872#gid=1575975872'

'https://docs.google.com/spreadsheets/d/1qx6nagvF8okAKvfO9BoHL2aZUZ8NOnz2/edit?gid=1575975872#gid=1575975872'

*--> Agora basta clicar no menu 'Ambiente de execução' e em 'Executar tudo' (ou CTR + F9), nesta ordem.*

#### 1. CONFIGURAÇÃO DO AMBIENTE

##### 1.1 Bibliotecas Python

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
import importlib.util

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [8]:
%%capture
# Lista de pacotes necessários
required_packages = ['gspread', 'pandas', 'gspread_dataframe', 'openpyxl']

# Verificar e instalar pacotes que não estão instalados
def install_missing_packages(packages):
    for package in packages:
        if importlib.util.find_spec(package) is None:
            print(f'⚠ Instalando {package}...')
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '--upgrade', package])
        else:
            print(f'✔ {package} já está instalado')

install_missing_packages(required_packages)

##### 1.2 Fonte dos dados

In [9]:
# Por ora, uma planilha xlsx no Google Drive (vide 0.4). Futuramente, vincular a um banco de dados, sistema ou API (definir).

In [10]:
# Definir o ID da planilha e da aba (worksheet)
sheet_id = '1qx6nagvF8okAKvfO9BoHL2aZUZ8NOnz2'
gid = '1575975872'  # ID da aba específica

# Construir a URL para baixar a planilha como um arquivo Excel (.xlsx)
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=xlsx&id={sheet_id}&gid={gid}'

# Ler a planilha diretamente no Pandas
df_dados_contabeis = pd.read_excel(url, engine='openpyxl')

##### 1.3 Função Índices e Contas Contábeis

In [11]:
def calcular_indices_contabeis(df):
    indices = {}

    ### RENTABILIDADE
    indices['Margem Líquida'] = df['r_Lucro_Liquido'] / df['r_Receita_Total']
    # Percentual da receita que sobra como lucro líquido.

    indices['ROA'] = df['r_Lucro_Liquido'] / df['p_Ativo_Total']
    # Retorno sobre ativos: eficiência global da operação.

    indices['ROE'] = df['r_Lucro_Liquido'] / df['p_Patrimonio_Liquido']
    # Retorno sobre o patrimônio líquido: rentabilidade para o acionista.

    ### EBITDA
    ebit = df['r_Lucro_Liquido'] + df['r_Despesa_de_Juros'] + df['r_Despesa_de_Impostos']
    amort = df.get('r_Amortizacao', 0).fillna(0)
    depr = df.get('r_Depreciacao', 0).fillna(0)
    ebitda = ebit + amort + depr
    indices['EBITDA'] = ebitda
    indices['Margem EBITDA'] = ebitda / df['r_Receita_Total']
    # Percentual da receita que vira caixa operacional antes de juros, impostos e depreciação.

    ### ALAVANCAGEM E ENDIVIDAMENTO
    df['p_Divida_Bruta'] = df['p_Passivo_Total'] - df['p_Patrimonio_Liquido']
    # Endividamento Bruto (passivo exigível)

    df['p_Divida_Liquida'] = df['p_Divida_Bruta'] - df['p_Caixa']
    # Dívida Líquida (ajustada pelo caixa disponível)

    indices['Alavancagem'] = df['p_Divida_Liquida'] / ebitda
    # Mede quantos anos de geração operacional (EBITDA) seriam necessários para quitar a dívida líquida.

    indices['Endividamento'] = df['p_Divida_Bruta'] / df['p_Ativo_Total']
    # Percentual dos ativos financiado exclusivamente por capital de terceiros.

    ### ESTRUTURA DE CAPITAL
    df['p_Imobilizado'] = df['p_Ativo_Total'] - df['p_Ativo_Circulante']
    # Estimativa do Ativo Imobilizado

    indices['Imobilizado/Ativo'] = df['p_Imobilizado'] / df['p_Ativo_Total']
    # Percentual do ativo total que está imobilizado (capital fixo).

    ### COBERTURA DE JUROS
    indices['Cobertura de Juros'] = ebit / df['r_Despesa_de_Juros']
    # Mede a capacidade da empresa de pagar seus juros com o lucro operacional (EBIT).

    ### EFICIÊNCIA OPERACIONAL
    indices['Giro do Ativo'] = df['r_Receita_Total'] / df['p_Ativo_Total']
    # Mede quantas vezes os ativos se transformam em receita no período.

    ### CICLO OPERACIONAL
    indices['Período Médio de Recebimento'] = df['p_Contas_a_Receber'] / df['r_Receita_Total'] * 365
    indices['Período Médio de Pagamento'] = df['p_Contas_a_Pagar'] / df['r_Custos'] * 365
    # Dias médios para receber e pagar.

    ### LIQUIDEZ
    indices['Liquidez Corrente'] = df['p_Ativo_Circulante'] / df['p_Passivo_Circulante']
    # Mede a capacidade de pagar obrigações de curto prazo.

    indices['Liquidez Seca'] = (df['p_Ativo_Circulante'] - df['p_Estoques']) / df['p_Passivo_Circulante']
    # Medida mais conservadora de liquidez, desconsidera estoques.

    indices['CCL/Ativo Total'] = (df['p_Ativo_Circulante'] - df['p_Passivo_Circulante']) / df['p_Ativo_Total']
    # Mede a folga de capital de giro em relação ao total de ativos da empresa.


    ### Criar DataFrame
    df_indices = pd.DataFrame(indices)

    ### Tratar divisões por zero e valores infinitos
    df_indices.replace([np.inf, -np.inf], np.nan, inplace=True)

    return df_indices


##### 1.4 Importação dos Dados Contábeis

In [12]:
## Importar os dados
# - Do mais recente para o mais antigo

# arquivo_dados_contabeis = '/content/dados_contabeis_global.xlsx'
arquivo_dados_contabeis = df_dados_contabeis

In [13]:
arquivo_dados_contabeis

,ano,p_Ativo_Circulante,p_Ativo_Total,p_Caixa,p_Estoques,p_Contas_a_Pagar,p_Contas_a_Receber,p_Passivo_Circulante,p_Passivo_Total,p_Patrimonio_Liquido,r_Amortizacao,r_Custos,r_Depreciacao,r_Despesa_de_Impostos,r_Despesa_de_Juros,r_Lucro_Liquido,r_Receita_Total
0,0,29716613,58240254,0,0,6048227,8920741,6279752,58240254,12106709,0,67838821,0,0,31879,100893,73372438
1,1,35030579,67992387,0,0,16076322,10463734,16514038,67992387,10874130,0,33665834,0,0,334490,1496210,49372166
2,2,37531910,80987685,0,0,19191518,7861523,25235400,80987685,14031963,0,32800463,0,0,1144121,3748957,45676317


##### 1.5 Leitura dos Dados Contábeis

In [14]:
df_dados_contabeis = arquivo_dados_contabeis
print('Dados Contabeis Importados:')
print(df_dados_contabeis)

Dados Contabeis Importados:
   ano  p_Ativo_Circulante  p_Ativo_Total  p_Caixa  p_Estoques  \
0    0            29716613       58240254        0           0   
1    1            35030579       67992387        0           0   
2    2            37531910       80987685        0           0   

   p_Contas_a_Pagar  p_Contas_a_Receber  p_Passivo_Circulante  \
0           6048227             8920741               6279752   
1          16076322            10463734              16514038   
2          19191518             7861523              25235400   

   p_Passivo_Total  p_Patrimonio_Liquido  r_Amortizacao  r_Custos  \
0         58240254              12106709              0  67838821   
1         67992387              10874130              0  33665834   
2         80987685              14031963              0  32800463   

   r_Depreciacao  r_Despesa_de_Impostos  r_Despesa_de_Juros  r_Lucro_Liquido  \
0              0                      0               31879           100893   
1        

#### 2. PROCESSAMENTO DO MODELO #####

##### 2.1 Índices contábeis

In [15]:
# Calculado com base nos dados contábeis

In [16]:
df_indices = calcular_indices_contabeis(df_dados_contabeis).round(2)
print('\nÍndices Contábeis Calculados:')


Índices Contábeis Calculados:


In [17]:
if (df_dados_contabeis['p_Estoques'] == 0).all():
        del df_indices['Liquidez Seca']

In [18]:
print(df_indices)

   Margem Líquida   ROA   ROE   EBITDA  Margem EBITDA  Alavancagem  \
0            0.00  0.00  0.01   132772           0.00       347.46   
1            0.03  0.02  0.14  1830700           0.04        31.20   
2            0.08  0.05  0.27  4893078           0.11        13.68   

   Endividamento  Imobilizado/Ativo  Cobertura de Juros  Giro do Ativo  \
0           0.79               0.49                4.16           1.26   
1           0.84               0.48                5.47           0.73   
2           0.83               0.54                4.28           0.56   

   Período Médio de Recebimento  Período Médio de Pagamento  \
0                         44.38                       32.54   
1                         77.36                      174.30   
2                         62.82                      213.56   

   Liquidez Corrente  CCL/Ativo Total  
0               4.73             0.40  
1               2.12             0.27  
2               1.49             0.15  


##### 2.2 Padronização dos índices contábeis

In [19]:
# Escalar os índices contábeis para o PCA
scaler = StandardScaler()
indices_scaled = scaler.fit_transform(df_indices)
print('\nÍndices Escalados para PCA:')
print(indices_scaled)


Índices Escalados para PCA:
[[-1.1111678  -1.13554995 -1.22474487 -1.09286499 -1.09985336  1.41267511
  -1.38873015 -0.50800051 -0.8061489   1.37535374 -1.27006502 -1.38383806
   1.39023519  1.24074398]
 [-0.20203051 -0.16222142  0.         -0.23089278 -0.21997067 -0.64922553
   0.9258201  -0.88900089  1.40935123 -0.40254256  1.1737357   0.43944297
  -0.47054114 -0.03265116]
 [ 1.31319831  1.29777137  1.22474487  1.32375777  1.31982404 -0.76344958
   0.46291005  1.3970014  -0.60320233 -0.97281118  0.09632932  0.94439509
  -0.91969405 -1.20809282]]


##### 2.3 Cálculo do PCA

In [20]:
# Calculado com base nos índices contábeis

In [21]:
# Realizar o PCA
pca = PCA()
pca_result = pca.fit_transform(indices_scaled)
print('\nComponentes Principais (PCA):')
print(pca_result)

#relatório



Componentes Principais (PCA):
[[ 4.40713052e+00 -8.54068358e-01 -3.46049193e-16]
 [-7.50506373e-01  2.36982746e+00 -3.46049193e-16]
 [-3.65662415e+00 -1.51575910e+00 -3.46049193e-16]]


##### 2.4 Variância Explicada PCA

In [22]:
# Variância explicada pelos componentes principais
explained_variance_ratio = pca.explained_variance_ratio_
print('\nVariância Explicada por Componente:')
print(explained_variance_ratio)

#relatório


Variância Explicada por Componente:
[7.94213319e-01 2.05786681e-01 8.55357458e-33]


##### 2.5 DataFrame PCA

In [23]:
# DataFrame com os componentes principais
pca_df = pd.DataFrame(pca_result, columns=[f'PC{i+1}' for i in range(pca_result.shape[1])])
print('\nMatriz de Componentes Principais:')
print(pca_df)

#relatório


Matriz de Componentes Principais:
        PC1       PC2           PC3
0  4.407131 -0.854068 -3.460492e-16
1 -0.750506  2.369827 -3.460492e-16
2 -3.656624 -1.515759 -3.460492e-16


##### 2.6 Matriz de cargas

In [24]:
# Obter a matriz de cargas
loadings = pd.DataFrame(
    pca.components_,
    columns=df_indices.columns,
    index=[f'PC{i+1}' for i in range(pca.components_.shape[0])]
)

print('Matriz de Cargas dos Componentes Principais:')
print(loadings)

# Identificar os índices mais significativos para cada PC
print('\nÍndices mais significativos por componente:')
for pc in loadings.index:
    print(f'\n{pc}:')
    print(loadings.loc[pc].abs().sort_values(ascending=False).head(3))  # Top 3 índices mais significativos

#relatório

Matriz de Cargas dos Componentes Principais:
     Margem Líquida       ROA       ROE    EBITDA  Margem EBITDA  Alavancagem  \
PC1       -0.286216 -0.288642 -0.296071 -0.284306      -0.285044     0.284940   
PC2       -0.175894 -0.159864 -0.093764 -0.187468      -0.183093    -0.183716   
PC3        0.424988  0.690044 -0.043079 -0.190974      -0.192743    -0.012181   

     Endividamento  Imobilizado/Ativo  Cobertura de Juros  Giro do Ativo  \
PC1      -0.255054          -0.200256           -0.072094       0.297410   
PC2       0.309896          -0.438553            0.571875      -0.075674   
PC3       0.157642          -0.157142            0.188303       0.254524   

     Período Médio de Recebimento  Período Médio de Pagamento  \
PC1                     -0.204769                   -0.296246   
PC2                      0.430434                    0.091614   
PC3                     -0.235951                    0.005078   

     Liquidez Corrente  CCL/Ativo Total  
PC1           0.295083

In [25]:
# Criar dicionário com top 3 índices por componente
top_indices_por_pc = {
    pc: loadings.loc[pc].abs().sort_values(ascending=False).head(3)
    for pc in loadings.index
}

# Transformar em DataFrame
top_indices_df = pd.DataFrame([
    {
        'PC': pc,
        'Indice 1': top.index[0],
        'Peso 1': round(top.values[0], 3),
        'Indice 2': top.index[1],
        'Peso 2': round(top.values[1], 3),
        'Indice 3': top.index[2],
        'Peso 3': round(top.values[2], 3)
    }
    for pc, top in top_indices_por_pc.items()
])

# Exibir ou salvar
print('\nTop 3 Índices por Componente Principal:')
print(top_indices_df)



Top 3 Índices por Componente Principal:
    PC            Indice 1  Peso 1           Indice 2  Peso 2  \
0  PC1       Giro do Ativo   0.297    CCL/Ativo Total   0.297   
1  PC2  Cobertura de Juros   0.572  Imobilizado/Ativo   0.439   
2  PC3                 ROA   0.690     Margem Líquida   0.425   

                       Indice 3  Peso 3  
0    Período Médio de Pagamento   0.296  
1  Período Médio de Recebimento   0.430  
2                 Giro do Ativo   0.255  


##### 2.7 Score PCA final

In [26]:
pca_df.dot(explained_variance_ratio)

0    3.324446
1   -0.108383
2   -3.216063
dtype: float64

#### 3. RESULTADOS #####

##### 3.1A Função Finscore Bruto

In [27]:
# Função para categorizar escores consolidados com mais granularidade
def categorias_finscore_bruto(escores):
    categorias = []
    for escore in escores:
        if escore > 1.5:
            categorias.append('Muito Abaixo do Risco')
        elif 1.0 < escore <= 1.5:
            categorias.append('Levemente Abaixo do Risco')
        elif -1.0 <= escore <= 1.0:
            categorias.append('Neutro')
        elif -1.5 < escore < -1.0:
            categorias.append('Levemente Acima do Risco')
        else:
            categorias.append('Muito Acima do Risco')
    return categorias

# Tabela de Categorias:
# -------------------------------------------------------
# |   Intervalo do Escore   |      Categoria            |
# -------------------------------------------------------
# |  escore > 1.5           | Muito Abaixo do Risco     |
# |  1.0 < escore ≤ 1.5     | Levemente Abaixo do Risco |
# | -1.0 ≤ escore ≤ 1.0     | Neutro                    |
# | -1.5 < escore < -1.0    | Levemente Acima do Risco  |
# |  escore ≤ -1.5          | Muito Acima do Risco      |
# -------------------------------------------------------

##### 3.1B Calcular Finscore Bruto

In [28]:
# Calcular o escore consolidado com penalização do último ano
# Do mais recente para o mais antigo: 
pesos = [0.6, 0.25, 0.15]  # Pesos para os três anos
finscore_bruto = round((pca_df.dot(explained_variance_ratio) * pesos).sum(), 2)
print('\nFINSCORE BRUTO:')
print(finscore_bruto)
print('\nCLASSIFICAÇÃO FINSCORE BRUTO:')
print(categorias_finscore_bruto([finscore_bruto])[0])



FINSCORE BRUTO:
1.49

CLASSIFICAÇÃO FINSCORE BRUTO:
Levemente Abaixo do Risco


##### 3.2A Função Finscore Ajustado

In [29]:
# Este é o finscore que será considerado para o relatório.

In [30]:
# Função para categorizar escores consolidados com mais granularidade
def categorias_finscore_ajustado(escore):
    if escore > 875:
        return 'Muito Abaixo do Risco'
    elif 750 < escore <= 875:
        return 'Levemente Abaixo do Risco'
    elif 250 <= escore <= 750:
        return 'Neutro'
    elif 125 < escore < 250:
        return 'Levemente Acima do Risco'
    else:
        return 'Muito Acima do Risco'


# Tabela de Categorias:
# -------------------------------------------------------
# |   Intervalo do Escore  |      Categoria            |
# -------------------------------------------------------
# |  escore > 750          | Muito Abaixo do Risco     |
# |  500 < escore ≤ 750    | Levemente Abaixo do Risco |
# | 250 ≤ escore ≤ 500     | Neutro                    |
# | 100 < escore < 250     | Levemente Acima do Risco  |
# |  escore ≤ 100          | Muito Acima do Risco      |
# -------------------------------------------------------

##### 3.2B Calcular Finscore Ajustado

In [31]:
# Traduz a tendência em um escore de risco ajustado ao ponto de referência atual do cliente.
finscore_ajustado = round(min(((finscore_bruto + 2)/4)*1000, 1000), 2)

# Converte escore bruto (-2 a +2) para uma escala de 0 a 1000, truncando em 1000.


In [32]:
print('\nFINSCORE AJUSTADO:')
print(finscore_ajustado)
print('\nCLASSIFICAÇÃO FINSCORE AJUSTADO:')
print(categorias_finscore_ajustado(finscore_ajustado))


FINSCORE AJUSTADO:
872.5

CLASSIFICAÇÃO FINSCORE AJUSTADO:
Levemente Abaixo do Risco


##### 3.3A Função Serasa

In [33]:
# Função para categorizar escores conforme a classificação do Serasa
def categorias_serasa(score):
    if score > 700:
        return 'Excelente'
    elif score > 500:
        return 'Bom'
    elif score > 300:
        return 'Baixo'
    else:
        return 'Muito Baixo'

# Tabela de Categorias:
# --------------------------------------
# |   Intervalo do Escore  | Categoria |
# --------------------------------------
# |  701 a 1.000          | Excelente  |
# |  501 a 700            | Bom        |
# |  301 a 500            | Baixo      |
# |  0 a 300              | Muito Baixo|
# --------------------------------------

##### 3.3B Calcular Serasa

In [34]:
print('\nSERASA:')
print(serasa)
print('\nCLASSIFICAÇÃO SERASA:')
print(categorias_serasa(serasa))


SERASA:
550

CLASSIFICAÇÃO SERASA:
Bom


##### 3.4 Valores e Contas Contábeis

In [35]:
# Apenas para revisão

In [36]:
df_dados_contabeis
# Sendo 0 o mais recente e 2 o mais antigo

,ano,p_Ativo_Circulante,p_Ativo_Total,p_Caixa,p_Estoques,p_Contas_a_Pagar,p_Contas_a_Receber,p_Passivo_Circulante,p_Passivo_Total,p_Patrimonio_Liquido,r_Amortizacao,r_Custos,r_Depreciacao,r_Despesa_de_Impostos,r_Despesa_de_Juros,r_Lucro_Liquido,r_Receita_Total,p_Divida_Bruta,p_Divida_Liquida,p_Imobilizado
0,0,29716613,58240254,0,0,6048227,8920741,6279752,58240254,12106709,0,67838821,0,0,31879,100893,73372438,46133545,46133545,28523641
1,1,35030579,67992387,0,0,16076322,10463734,16514038,67992387,10874130,0,33665834,0,0,334490,1496210,49372166,57118257,57118257,32961808
2,2,37531910,80987685,0,0,19191518,7861523,25235400,80987685,14031963,0,32800463,0,0,1144121,3748957,45676317,66955722,66955722,43455775


##### 3.5 Índices contábeis

In [37]:
# Apenas para revisão

In [38]:
print(df_indices)
# Sendo 0 o mais recente e 2 o mais antigo


   Margem Líquida   ROA   ROE   EBITDA  Margem EBITDA  Alavancagem  \
0            0.00  0.00  0.01   132772           0.00       347.46   
1            0.03  0.02  0.14  1830700           0.04        31.20   
2            0.08  0.05  0.27  4893078           0.11        13.68   

   Endividamento  Imobilizado/Ativo  Cobertura de Juros  Giro do Ativo  \
0           0.79               0.49                4.16           1.26   
1           0.84               0.48                5.47           0.73   
2           0.83               0.54                4.28           0.56   

   Período Médio de Recebimento  Período Médio de Pagamento  \
0                         44.38                       32.54   
1                         77.36                      174.30   
2                         62.82                      213.56   

   Liquidez Corrente  CCL/Ativo Total  
0               4.73             0.40  
1               2.12             0.27  
2               1.49             0.15  


#### 4. PREPARAÇÃO IA

In [39]:
# Prompts e funções para o ChatGPT e LangChain

##### 4.1 Dicionário IA

In [40]:
# Criar o DataFrame com os resultados principais para IA
resultados_df_final = pd.DataFrame({
    'Métrica': ['Finscore', 'Serasa'],
    'Valor': [finscore_ajustado, serasa],
    'Categoria': [
        categorias_finscore_ajustado(finscore_ajustado),
        categorias_serasa(serasa)
    ]
})


In [41]:
# Função para preparar os dados em formato de dicionário para a IA
def preparar_para_llm(df_indices, resultados_df_final, top_indices_df, df_dados_contabeis):
    return {
        'cliente': cliente,
        'periodo_analise': f'{ano_inicial}–{ano_final}',
        'indices_financeiros': df_indices.to_dict(orient='records'),
        'finScore_resultado': resultados_df_final.to_dict(orient='records'),
        'pca_destaques': top_indices_df.to_dict(orient='records'),
        'dados_contabeis': df_dados_contabeis.to_dict(orient='records')
    }


##### 4.2 Chamar dados IA

In [42]:
# Chamada da função com os dados disponíveis
contexto_ia = preparar_para_llm(
    df_indices=df_indices,
    resultados_df_final=resultados_df_final,
    top_indices_df=top_indices_df,
    df_dados_contabeis=df_dados_contabeis
)

##### 4.3 Visualizar Dados IA

In [43]:
# Visualização opcional do dicionário gerado (útil para debug ou checagem)
import json
print('\n📦 DADOS PREPARADOS PARA IA (formato .json simplificado):')
print(json.dumps(contexto_ia, indent=2, ensure_ascii=False)[:3000])  # mostra só os primeiros 3000 caracteres


📦 DADOS PREPARADOS PARA IA (formato .json simplificado):
{
  "cliente": "CARGOBR TRANSPORTES",
  "periodo_analise": "2021–2023",
  "indices_financeiros": [
    {
      "Margem Líquida": 0.0,
      "ROA": 0.0,
      "ROE": 0.01,
      "EBITDA": 132772,
      "Margem EBITDA": 0.0,
      "Alavancagem": 347.46,
      "Endividamento": 0.79,
      "Imobilizado/Ativo": 0.49,
      "Cobertura de Juros": 4.16,
      "Giro do Ativo": 1.26,
      "Período Médio de Recebimento": 44.38,
      "Período Médio de Pagamento": 32.54,
      "Liquidez Corrente": 4.73,
      "CCL/Ativo Total": 0.4
    },
    {
      "Margem Líquida": 0.03,
      "ROA": 0.02,
      "ROE": 0.14,
      "EBITDA": 1830700,
      "Margem EBITDA": 0.04,
      "Alavancagem": 31.2,
      "Endividamento": 0.84,
      "Imobilizado/Ativo": 0.48,
      "Cobertura de Juros": 5.47,
      "Giro do Ativo": 0.73,
      "Período Médio de Recebimento": 77.36,
      "Período Médio de Pagamento": 174.3,
      "Liquidez Corrente": 2.12,
      "

In [44]:
# A versão com llm.invoke é ótima para validar o texto direto.

# A versão com LLMChain vai te preparar para expandir depois (ex: exportar para Word, gerar resumo técnico, enviar via API.

#### 5. PARECER IA

In [45]:
# https://platform.openai.com/docs/overview
# https://platform.openai.com/settings/organization/billing/history
# https://platform.openai.com/settings/organization/usage

##### 5.1 Promp Finscore

In [46]:
from langchain.prompts import PromptTemplate

prompt_template = """
Você é um analista de crédito sênior de um banco de desenvolvimento. Recebeu os dados financeiros consolidados de uma empresa, contendo contas contábeis, índices financeiros, componentes principais (PCA) e os escores de risco FinScore e Serasa.

Com base nesses dados, elabore um **parecer técnico completo** com os seguintes critérios:

1. **INTRODUÇÃO**
- Apresente o objetivo do parecer.
- Identifique o cliente, o período da análise e o escopo da avaliação (contábil, econômico-financeira e de risco).

2. **METODOLOGIA**
- Explique o que é o FinScore, como é calculado (PCA + índices contábeis) e sua utilidade para análise de crédito.
- Explique o Serasa Score como índice complementar e relacione ambos no contexto do diagnóstico de risco.
- Diga que os índices foram padronizados e ponderados no tempo, destacando os principais vetores do PCA.

3. **ANÁLISE DAS CONTAS CONTÁBEIS**
- Apresente uma **tabela com os valores principais** (ativos, passivos, lucro, receitas, caixa etc.).
- Comente **criticamente a evolução** desses valores ao longo do tempo, apontando **pontos fortes, riscos ou deteriorações**.

4. **ANÁLISE DOS ÍNDICES FINANCEIROS**
- Mostre uma **tabela com todos os índices calculados**.
- Comente um a um: liquidez, rentabilidade, alavancagem, giro, cobertura de juros, prazos médios etc.
- Destaque **os três mais relevantes** segundo o PCA e interprete **por que eles se destacam** e o que indicam.

5. **FINSCORE E SERASA SCORE**
- Apresente os dois scores obtidos, em formato **tabela com classificação**.
- Comente **as classificações e coerência entre os escores**.
- Relacione os escores com os dados contábeis e com a saúde financeira geral da empresa.

6. **CONCLUSÃO E VEREDICTO FINAL**
- Diga se recomenda a concessão de crédito (sim, não, sim com ressalvas).
- Fundamente a decisão com base em dados objetivos e análises anteriores.
- Seja **criterioso e técnico**, considerando o risco vs. retorno para a instituição.

7. **RECOMENDAÇÕES PARA MITIGAÇÃO DE RISCOS**
- Sugira garantias, limites prudenciais, cláusulas contratuais, acompanhamento contínuo do FinScore e envio trimestral de dados financeiros.
- Se necessário, proponha exigência de avalistas ou reforço de capital.

**IMPORTANTE:**  
- Escreva com clareza, formalidade e linguagem técnica.
- Insira as **tabelas e gráficos onde for pertinente** (mesmo que simbólicos, como: [Tabela: Índices Financeiros], [Gráfico: Evolução do Endividamento]).
- Evite redundâncias. Destaque **o que os números revelam**.
- Apresente **insights concretos** para o tomador de decisão.

DADOS DISPONÍVEIS:
- Cliente: {cliente}
- Período da análise: {periodo_analise}
- Contas contábeis: {dados_contabeis}
- Índices financeiros: {indices_financeiros}
- FinScore e Serasa: {finScore_resultado}
- PCA (destaques): {pca_destaques}
"""

# Criar o PromptTemplate
prompt = PromptTemplate.from_template(prompt_template)


##### 5.2 Chave OpenAI

In [49]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os

# Carregar variáveis do .env
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
print(f'API_KEY carregada: {api_key is not None}')

# Verificar se a chave foi carregada corretamente
if api_key:
    print('OPENAI_API_KEY carregada com sucesso.')
else:
    print('OPENAI_API_KEY não encontrada. Verifique seu arquivo .env.')

API_KEY carregada: False
OPENAI_API_KEY não encontrada. Verifique seu arquivo .env.


In [53]:
print("Diretório atual:", os.getcwd())

Diretório atual: C:\Users\ferna\AppData\Local\Programs\Microsoft VS Code


##### 5.3 Configurar modelo OpenAI com LangChain

In [48]:
from langchain_community.chat_models import ChatOpenAI

# Escolher o modelo a ser utilizado
modelo = 4 # '3.5' ou '4'

# ifelse modelo gpt-3.5 ou gpt-4
if modelo == 3.5:
    model = 'gpt-3.5-turbo'
elif modelo == 4:
    model = 'gpt-4'
else:
    raise ValueError('Modelo inválido. Escolha entre 3.5 ou 4.')

# Cria uma instância do modelo ChatGPT da OpenAI
llm = ChatOpenAI(
    temperature=0.2,       # Baixa temperatura → respostas mais objetivas e determinísticas
    model=model,           # Usando o modelo definido acima
    openai_api_key=api_key
)

print('Modelo ChatOpenAI configurado com sucesso.')


C:\Users\ferna\AppData\Local\Temp\ipykernel_31316\1921679759.py:15: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'temperature': 0.2, 'ope...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

##### 5.4 Gerar o relatório

In [ ]:
prompt_final = prompt.format(**contexto_ia)
resposta = llm.invoke(prompt_final)
print(resposta.content) 


**PARECER TÉCNICO DE ANÁLISE DE CRÉDITO**

**1. INTRODUÇÃO**

Este parecer tem como objetivo avaliar a concessão de crédito para a empresa CARGOBR TRANSPORTES. A análise abrange o período de 2021 a 2023 e contempla a avaliação contábil, econômico-financeira e de risco.

**2. METODOLOGIA**

O FinScore é um índice de risco de crédito que combina a análise de componentes principais (PCA) com índices contábeis. Ele é útil para avaliar a capacidade de uma empresa em honrar suas obrigações financeiras. O Serasa Score é um índice complementar que avalia o histórico de crédito da empresa. Ambos os índices são relevantes para o diagnóstico de risco.

Os índices foram padronizados e ponderados no tempo, com destaque para os principais vetores do PCA: Giro do Ativo, CCL/Ativo Total e Período Médio de Pagamento.

**3. ANÁLISE DAS CONTAS CONTÁBEIS**

[Tabela: Contas Contábeis]

A empresa apresentou um aumento progressivo do Ativo Total e do Passivo Total ao longo do período analisado. O Lucro Líqui

In [ ]:
# Fazer exportação word